# Vllm

<center>
<img src="https://docs.vllm.ai/en/stable/assets/logos/vllm-logo-text-light.png" alt="vllm logo" width="400"/>
</center>

Vllm is less cool and has an ugly logo :) 

### 🏳️ Open the cc-in2p3 Jupyter Notebook launcher at 

> `https://notebook.cc.in2p3.fr`


launch a new terminal session

<center>
    <img src="https://icon-library.com/images/bash-icon/bash-icon-6.jpg" alt="bash is also cool" width="200"/>
</center>


and execute therein the following commands to load the previously prepared enviroment with vllm installed

```bash
# Set up Modules
source /etc/profile.d/modules.sh
module purge
module load Programming_Languages/python/3.11.4

# Activate conda environment
source /pbs/throng/training/universite-hiver/wschooloptim/vllm/bin/activate

# Enforce correct site-packages
unset PYTHONPATH
export PYTHONPATH=/pbs/throng/training/universite-hiver/wschooloptim/vllm/lib/python3.11/site-packages/

# Ollama and HF cache setting
export HUGGINGFACE_HUB_CACHE=/pbs/throng/training/universite-hiver/cache/huggingface
export OLLAMA_MODELS=/pbs/throng/training/universite-hiver/cache/ollama/models
```

launch an vllm server

```bash
vllm serve HuggingFaceH4/zephyr-7b-beta --max_model_len=10000 --guided_decoding_backend=xgrammar --seed=1  --tensor-parallel-size 1 &
```

Create one request to ollama's server:

```bash
curl -X POST http://localhost:8000/v1/chat/completions \
-H "Content-Type: application/json" \
-d '{
  "model": "HuggingFaceH4/zephyr-7b-beta",
  "messages": [{ "role": "user", "content": "Who won the last presidential elections in France?" }]
}'
```

Keep that terminal open for the rest of the tutorial, we will check therein ollama server logs.

### 🏴 Open another cc-in2p3 Jupyter Notebook launcher at 

> `https://notebook.cc.in2p3.fr`


and launch now the Notebook called `Python 3.13 (Ollama)`.

We will work from now in that notebook.

In [ ]:
! xan tail /pbs/home/u/univ-hiver01/private/resinwinterschool/tweets_spanish_english_25K.csv | /pbs/throng/training/universite-hiver/wschooloptim/vllm/bin/xan v 

##### Import python packages

In [ ]:
import csv
import time
import asyncio
from openai import OpenAI
from string import Template
import nest_asyncio
nest_asyncio.apply()

##### Load inputs and create asynchronous iterator

In [ ]:
input_file = "/pbs/home/u/univ-hiver01/private/resinwinterschool/tweets_spanish_english_25K.csv"
content_column = 'english'

start = time.time()
with open(input_file, 'r') as f:
    tweets = [d[content_column] for d in csv.DictReader(f)]
end = time.time()
print(f"Loading {len(tweets)} tweets tooks {end - start} seconds.")


##### Load choices and extra body dictionary with structured_outputs options

In [ ]:
choices_file = "/pbs/throng/training/universite-hiver/wschooloptim/resinwinterschool/multiple.txt"
with open(choices_file, 'r') as f:
    choice = f.read().split("\n")

extra_body = {"structured_outputs": {"choice": choice}}

print(extra_body)

##### Load instructions

In [ ]:
instructions_file = "/pbs/throng/training/universite-hiver/wschooloptim/resinwinterschool/instructions.txt"
with open(instructions_file, 'r') as f:
    instructions = f.read()
print(instructions)

##### Create vllm openai client

In [ ]:
client = OpenAI(
    base_url="http://localhost:8000/v1", # this is the same HOST given at ollama server launching
    api_key="EMPTY"  # required but unused
)

##### Create async functions to asynchronously request llm server


In [ ]:
async def inputsIterator(start, end):
    for tweet in tweets[start:end]:
        yield tweet

In [ ]:
async def doCompletetion(tweet):
    return client.chat.completions.create(
        model=client.models.list().data[0].id,
        messages=[{
            'role': 'user',
            'content': Template(instructions).substitute(tweet=tweet)
        }],
        extra_body=extra_body,
        max_completion_tokens=16,
        temperature=0.7,
        top_p=0.95,
    )

In [ ]:
async def run_all(start, end):
    # Asynchronously call the function for each prompt
    tasks = [
        doCompletetion(tweet)
        async for tweet in inputsIterator(start, end)
    ]
    # Gather and run the tasks concurrently
    results = await asyncio.gather(*tasks)
    return results

In [ ]:
# Run all courutines
init_idx = 3449
nb_tweets = 100

start = time.time()
results = asyncio.run(run_all(start=init_idx, end=init_idx + nb_tweets))
elapsed = time.time() - start

whole_db_nb_tweets = int(1e6)


print(f"""
Annotationg {nb_tweets} tweets took {elapsed} seconds,
this is {elapsed / nb_tweets} seconds per tweet or
{(1 / (24 * 3600)) * whole_db_nb_tweets * elapsed / nb_tweets} days for the whole database of {whole_db_nb_tweets} inputs.
""")

> Annotationg 500 tweets took 27.607022047042847 seconds,
>
> tthis is 0.055214044094085694 seconds per tweet or
>
> 0.6670885399377953 days for the whole database of 1043873 inputs.

In [ ]:
results[:3]

In [ ]:
for tweet, answer in zip(tweets, [r.choices[0].message.content for r in results]):
    if answer == "Jara":
        print("-----------------------------------------------" + "\n" + tweet + "\n" + answer)